和波士顿房价预测相比，完成以下作业
2. 挑战题：用代码实现两层的神经网络的梯度传播，中间层的尺寸为13【房价预测案例】（教案当前的版本为一层的神经网络），如 **图14** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/580f2553aa4643809006f5a8d3deb2aa8dd4e1aa69d94cf6a35ead5fe7cf469e" width="300"  ></center>
<center><br>图14：两层的神经网络</br></center>
<br></br>

看这里就行[main](#main)

In [ ]:
# 导入需要用到的package
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib



In [ ]:
# 读入训练数据
datafile = './work/housing.data'
data = np.fromfile(datafile, sep=' ')
data.shape

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows=None

## pandas读取

In [ ]:
# 导入数据 pandas
import numpy as np
import json
import pandas as pd
# 读入之后的数据被转化成1维array，其中array的第0-13项是第一条数据，第14-27项是第二条数据，以此类推.... 
# 这里对原始数据做reshape，变成N x 14的形式
feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE','DIS', 
                 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
#原来的数据不是用\t，一个或多个空格
old_df=pd.read_table('./work/housing.txt',names=feature_names)
old_df.head(3)
old_df[['DIS','TAX']]
old_df.max()
old_df.min()
df=(old_df-old_df.min())/(old_df.max()-old_df.min())
df.head(3)

In [ ]:
df.values.shape

### pandas划分训练和测试

In [ ]:
import paddle
import paddle.fluid as fluid
from sklearn.model_selection import train_test_split
import paddle.fluid.dygraph as dygraph
from paddle.fluid.dygraph import Linear
train_data,eval_data=train_test_split(df,test_size=0.1,random_state=42)
BATCH_SIZE = 16
def reader_creator2(data):
    def reader():
        for row in data.values:           
            yield [row[:-1], row[-1:]]          #返回Iterable对象
    return reader
train_reader = paddle.batch(reader_creator2(train_data),
                            batch_size=BATCH_SIZE)
test_reader = paddle.batch(reader_creator2(eval_data), 
                           batch_size=BATCH_SIZE)    

In [ ]:
train_data.head(3)
eval_data.head(3)

### 封装成load data函数

将上述几个数据处理操作封装成`load data`函数，以便下一步模型的调用，实现方法如下。

In [ ]:
def load_data():
    # 从文件导入数据
    datafile = './work/housing.data'
    data = np.fromfile(datafile, sep=' ')
    data

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算训练集的最大值，最小值，平均值    
    maximums, minimums, avgs = data.max(axis=0), data.min(axis=0), \
                                 data.sum(axis=0) / data.shape[0]

    # 对数据进行归一化处理
    for i in range(feature_num):
        print(feature_names[i],maximums[i], minimums[i], avgs[i])
        data[:, i] = (data[:, i] - minimums[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

In [ ]:
# 获取数据
training_data, test_data = load_data()
x = training_data[:, :-1]
y = training_data[:, -1:]

In [ ]:
# 查看数据
print(x[0])
print(y[0])
print(x[1])

In [ ]:
class Network(object):
    def __init__(self, num_of_weights):
        # 随机产生w的初始值
        # 为了保持程序每次运行结果的一致性，
        # 此处设置固定的随机数种子
        np.random.seed(0)
        self.w = np.random.randn(num_of_weights, 1)
        self.b = 0.
        
    def forward(self, x):
        z = np.dot(x, self.w) + self.b
        return z

基于Network类的定义，模型的计算过程如下所示。

In [ ]:
net = Network(13)
x1 = x[0]
y1 = y[0]
z = net.forward(x1)
print(z)

## 训练配置

模型设计完成后，需要通过训练配置寻找模型的最优值，即通过损失函数来衡量模型的好坏。训练配置也是深度学习模型关键要素之一。

通过模型计算$x_1$表示的影响因素所对应的房价应该是$z$, 但实际数据告诉我们房价是$y$。这时我们需要有某种指标来衡量预测值$z$跟真实值$y$之间的差距。对于回归问题，最常采用的衡量方法是使用均方误差作为评价模型好坏的指标，具体定义如下：

$$Loss = (y - z)^2$$

上式中的$Loss$（简记为: $L$）通常也被称作损失函数，它是衡量模型好坏的指标。在回归问题中，均方误差是一种比较常见的形式，分类问题中通常会采用交叉熵作为损失函数，在后续的章节中会更详细的介绍。对一个样本计算损失函数值的实现如下：

In [ ]:
Loss = (y1 - z)*(y1 - z)
print(Loss)

因为计算损失函数时需要把每个样本的损失函数值都考虑到，所以我们需要对单个样本的损失函数进行求和，并除以样本总数$N$。
$$Loss= \frac{1}{N}\sum_{i=1}^N{(y_i - z_i)^2}$$
在Network类下面添加损失函数的计算过程如下：

$$\frac{\partial{L}}{\partial{w_0}} = ({x_1^{0}\cdot w_0} + {x_1^{1}\cdot w_1} + ...  + {x_1^{12}\cdot w_{12}} + b - y_1)\cdot x_1^{0}=({z_1} - {y_1})\cdot x_1^{0}$$

$w_0$只和$x^{0}$有关这点很重要

$$dw=np.dot(x^{T},z-y)$$

$$x^{T}= \left\{\begin{matrix} x_1^{0} & x_2^{0} & x_3^{0}\\ x_1^{1} & x_2^{1} & x_3^{1} \\ \vdots & \vdots & \vdots \\ x_1^{12} & x_2^{12} & x_3^{12} \end{matrix} \right\}$$
$$z-y= \left\{\begin{matrix} z_1-y_1\\ z_2-y_2 \\ z_3-y_3 \end{matrix} \right\}$$
$$
x^{T}\cdot(z-y)=\left\{\begin{matrix} x_1^{0}\cdot(z_1-y_1)+ x_2^{0}\cdot(z_2-y_2) +x_3^{0}\cdot(z_3-y_3)\\ x_1^{1}\cdot(z_1-y_1)+ x_2^{2}\cdot(z_2-y_2) +x_3^{3}\cdot(z_3-y_3) \\ \vdots  \\ x_1^{12}\cdot(z_1-y_1)+ x_2^{12}\cdot(z_2-y_2) +x_3^{12}\cdot(z_3-y_3) \end{matrix} \right\}
$$

$$\frac{\partial{L}}{\partial{w_j}} = \frac{1}{N}\sum_{i=1}^N{(z_i - y_i)\frac{\partial{z_i}}{\partial{w_j}}} = \frac{1}{N}\sum_{i=1}^N{(z_i - y_i)x_i^{j}}$$

In [ ]:
class Network2(object):
    def __init__(self,num_of_weights):
        np.random.seed(100)
        self.w=np.random.randn(num_of_weights,1)
        self.b=0
    def forward(self,x):
        z=np.dot(x,self.w)+self.b
        return z
    def loss(self,z,y):
        error=z-y
        cost=error**2
        cost=np.mean(cost)
        return cost        
    def update(self,x,y,z):
        lr=0.01
        m=len(z)      
        dw=np.dot(x.T,z-y)/m
        #print(dw)
        self.w=self.w-lr*dw
        db=np.sum(z-y,axis=0)/m
        self.b=self.b-lr*db
    def train(self,x,y):
        z=self.forward(x)
        loss_=self.loss(z,y)
        self.update(x,y,z) 
        return loss_       

In [ ]:
net=Network2(13)
x_=x[:3]
y_=y[:3]
z=net.forward(x_)
cost=net.loss(y_,z)

z.shape
x_.shape
cost
net.update(x_,y_,z)
z=net.forward(x_)
cost=net.loss(y_,z)
cost

In [ ]:
#自己写的训练
from visualdl import LogWriter
writer=LogWriter('log/network2')
net=Network2(13)
losses=[]
for i in range(1000):
    loss_=net.train(x,y)
    if i%5==0 or i==999:print(loss_)

    writer.add_scalar('loss',step=i,value=loss_)    
    losses.append(loss_)

    if loss_<1e-4:break    
 

In [ ]:
#自己写的绘图
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

t=np.arange(0,1000,1)
fig,ax=plt.subplots()

ax.plot(t,losses)   

In [ ]:
#自己写的训练，加上batch
from visualdl import LogWriter
writer=LogWriter('log/network2-batch')
net=Network2(13)
losses=[]
batch_=32
np.random.shuffle(training_data)
batches_=[training_data[k:k+batch_] for k in range(0,len(training_data),batch_)]

print(batches_[0].shape)
print(batches_[-1].shape)
all_iter=0
for i in range(500):
    avg_loss=0
    for batch_id,mini_batch in enumerate(batches_):        
        x_=mini_batch[:,:-1]
        y_=mini_batch[:,-1:]
        loss_=net.train(x_,y_)
        avg_loss=avg_loss+loss_
        all_iter=all_iter+1
        if i%5==0 :print(loss_)
        writer.add_scalar('loss-32-e500',step=all_iter,value=loss_)    
        losses.append(loss_)
    avg_loss=avg_loss/len(training_data)
    print(avg_loss)
    if avg_loss<1e-4:break 

In [ ]:
def draw_process(iters,data,title='loss',color='green',label='loss'):
    plt.title(title, fontsize=24)
    plt.xlabel("iter", fontsize=20)
    plt.ylabel(label, fontsize=20)
    plt.plot(iters, data,color=color,label=label) 
    plt.legend()
    plt.grid()
    plt.show()

<span id="main"/>

$x->h1->z$，和真实值$y$
$$h1=w_1x+b_1,z=w_2h1+b_2$$
$$\frac{\partial{L}}{\partial{w_2}} = \frac{\partial{L}}{\partial{z}}\cdot \frac{\partial{z}}{\partial{w_2}}=\frac{\partial{L}}{\partial{z}}\cdot h1$$
$$\frac{\partial{L}}{\partial{b_2}} = \frac{\partial{L}}{\partial{z}}\cdot \frac{\partial{z}}{\partial{b_2}}=\frac{\partial{L}}{\partial{z}}\cdot 1$$
$$\frac{\partial{z}}{\partial{h1}} = w_2$$
$$\frac{\partial{L}}{\partial{w_1}} = \frac{\partial{L}}{\partial{z}}\cdot \frac{\partial{z}}{\partial{h1}} \cdot \frac{\partial{h1}}{\partial{w_1}}=\frac{\partial{L}}{\partial{z}}\cdot w_2 \cdot x$$
$$\frac{\partial{L}}{\partial{b_1}} = \frac{\partial{L}}{\partial{z}}\cdot \frac{\partial{z}}{\partial{h1}} \cdot \frac{\partial{h1}}{\partial{b_1}}=\frac{\partial{L}}{\partial{z}}\cdot w_2 \cdot 1$$
$$\frac{\partial{L}}{\partial{w_1}} = \frac{\partial{L}}{\partial{z}}\cdot \frac{\partial{z}}{\partial{h1}} \cdot \frac{\partial{h1}}{\partial{w_1}}$$
$$\frac{\partial{L}}{\partial{z}} = \frac{1}{N}\sum_{i=1}^N{(z_i - y_i)}$$

In [ ]:
#加个中间13层的网，13->13->1
import numpy as np
class Network13_13(object):
    def __init__(self,num_of_weights):
        np.random.seed(100)
        self.w1=np.random.randn(num_of_weights,num_of_weights)
        self.b1=np.random.randn(1,num_of_weights)
        self.w2=np.random.randn(num_of_weights,1)
        self.b2=0
    def forward(self,x):            
        h1=np.dot(x,self.w1)+self.b1
        z=np.dot(h1,self.w2)+self.b2
        #print(h1,z)
        return h1,z
    def loss(self,z,y):
        error=z-y
        cost=error**2
        cost=np.mean(cost)
        return cost                 
    def update(self,x,h1,z,y):#x->h1->z，实际值为y
        lr=0.0005
        m=len(y)      
        db2=np.sum(z-y,axis=0)/m
        dw2=np.dot(h1.T,z-y)/m
        dh1=np.dot(z-y,self.w2.T)/m
        db1=np.sum(dh1,axis=0)
        #db1=np.sum(self.w2,axis=0)        
        dw1=np.dot(x.T,dh1)
        """
        print('x.shape {},h1.shape {},z.shape {},y.shape{}'.format(x.shape,h1.shape,z.shape,y.shape))
        print('dw2',dw2.shape)
        print('db2',db2.shape)
        print('dw1',dw1.shape)        
        print('db1',db1.shape)
        """
        self.w1=self.w1-lr*dw1
        self.b2=self.b2-lr*db2
        self.b1=self.b1-lr*db1
        self.w2=self.w2-lr*dw2
    def train(self,x,y):
        h1,z=self.forward(x)    
        loss_=self.loss(z,y)
        self.update(x,h1,z,y)
        return loss_



net=Network13_13(13)

batch_=32
np.random.shuffle(training_data)
batches_=[training_data[k:k+batch_] for k in range(0,len(training_data),batch_)]

print(batches_[0].shape)
print(batches_[-1].shape)
losses=[]
all_iter=0
all_iters=[]
for i in range(50):
    avg_loss=0
    for batch_id,mini_batch in enumerate(batches_):        
        x_=mini_batch[:,:-1]
        y_=mini_batch[:,-1:]
        loss_=net.train(x_,y_)
        avg_loss=avg_loss+loss_
        all_iter=all_iter+1
        all_iters.append(all_iter)
        if i%5==0 :print(loss_)        
        losses.append(loss_)
    avg_loss=avg_loss/len(training_data)
    print('avg_loss',avg_loss)



In [ ]:

draw_process(all_iters,losses)